In [4]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

X = np.load('x_letters.npy')
y = np.load('y_letters.npy')

X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)

y_tensor = tf.convert_to_tensor(y, dtype=tf.float32)

y_flat = tf.reshape(y_tensor, [-1])

vals, idx, counts = tf.unique_with_counts(y_flat)

first_indices = []
for v in vals.numpy():
    pos = tf.where(y_flat == v)
    first_indices.append(pos[0][0].numpy())

# for i, val, count in zip(first_indices, vals.numpy(), counts.numpy()):
#     print(f"Value: {val}, Count: {count}")
#     img = X[i]
#     #plt.imshow(img, cmap='gray')
#     plt.axis('off')
#     plt.show()

In [5]:
from tensorflow.keras import layers, Sequential


class Descriminator(tf.keras.Model):
    def __init__(self):
        super(Descriminator, self).__init__()
        self.dense = layers.Dense(256, activation="relu")
        self.out_real_classifier = layers.Dense(1)
        self.out_label_classifier = layers.Dense(26)

    def call(self, x):
        x = self.dense(x)
        fake_logits = self.out_real_classifier(x)
        label_logits = self.out_label_classifier(x)
        return fake_logits, label_logits


class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.net = Sequential([
            layers.Dense(14 * 14, activation="relu"),
            layers.Dense(28 * 28, activation="sigmoid")])

    def call(self, noise, labels):
        x = tf.concat([noise, labels], axis=1)
        x = self.net(x)
        return x


def getDescriminatorLoss(real_reality_logits, fake_reality_logits, labels_logits, labels_gt):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    sce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    real_loss = bce(tf.ones_like(real_reality_logits), real_reality_logits)
    fake_loss = bce(tf.zeros_like(fake_reality_logits), fake_reality_logits)
    label_loss = sce(labels_gt, labels_logits)

    total_loss = real_loss + fake_loss + label_loss
    return total_loss


def getGeneratorLoss(fake_logits, labels_logits, labels_gt):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    sce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    real_loss = bce(tf.ones_like(fake_logits), fake_logits)
    label_loss = sce(labels_gt, labels_logits)

    total_loss = real_loss + label_loss
    return total_loss

In [12]:
import keras
from keras import layers
from keras import ops


class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.seed_generator = keras.random.SeedGenerator(1337)

    def compile(self, d_optimizer, g_optimizer, loss_fn_discriminator, loss_fn_generator):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn_generator = loss_fn_generator
        self.loss_fn_discriminator = loss_fn_discriminator
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        #sample random points in the latent space
        batch_size = ops.shape(real_images)[0]
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim)
        )

        generated_images = self.generator(random_latent_vectors)

        combined_images = ops.concatenate([generated_images, real_images], axis=0)

        labels = ops.concatenate(
            [ops.ones((batch_size, 1)), ops.zeros((batch_size, 1))], axis=0
        )

        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        with tf.GradientTape() as tape:
            reality_logits, label_logits = self.discriminator(combined_images)
            d_loss = self.loss_fn_discriminator(labels, predictions)       
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        #sample random points in latent space
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim)
        )

        #misleading labels saying "all real"
        misleading_labels = ops.zeros((batch_size, 1))

        #train generator (don't touch the weights of the discriminator)
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn_generator(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        #update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
